# Doel van dit notebook
In dit experiment onderzoek ik of een Conv1D-laag vóór een GRU helpt bij gesture-classificatie.
Hypothese: lokale temporele patronen (acceleratie-pieken) kunnen door Conv1D beter worden geëxtraheerd voordat de RNN de sequentie samenvat.
Ik vergelijk dit model met de eerdere GRU + mean-pooling baseline, met identieke data- en trainingsinstellingen.

In [1]:
import sys
from pathlib import Path

# Vind repo-root door omhoog te lopen totdat we pyproject.toml of .git vinden
p = Path().resolve()
while not (p / "pyproject.toml").exists() and not (p / ".git").exists():
    if p.parent == p:
        raise RuntimeError("Repo root not found (no pyproject.toml/.git).")
    p = p.parent

PROJECT_ROOT = p
sys.path.insert(0, str(PROJECT_ROOT))

print("Project root added to sys.path:", PROJECT_ROOT)

Project root added to sys.path: /home/mischatrader1/MADS/MADS-MachineLearning-course


In [2]:
from pathlib import Path

import torch

from mads_datasets import DatasetFactoryProvider, DatasetType
from mltrainer.preprocessors import PaddedPreprocessor

from mltrainer import TrainerSettings, ReportTypes
from mltrainer.metrics import Accuracy

In [3]:
import torch

if torch.backends.mps.is_available() and torch.backends.mps.is_built():
    device = torch.device("mps")
    print("Using MPS")
elif torch.cuda.is_available():
    device = torch.device("cuda:0")
    print("Using CUDA")
else:
    device = torch.device("cpu")
    print("Using CPU")

# Override naar CPU voor kleine RNN-modellen indien gewenst
device = torch.device("cpu")

device

Using CPU


/home/mischatrader1/MADS/MADS-MachineLearning-course/.venv/lib/python3.11/site-packages/torch/cuda/__init__.py:174: UserWarning: CUDA initialization: The NVIDIA driver on your system is too old (found version 11070). Please update your GPU driver by downloading and installing a new version from the URL: http://www.nvidia.com/Download/index.aspx Alternatively, go to: https://pytorch.org to install a PyTorch version that has been compiled with your version of the CUDA driver. (Triggered internally at /pytorch/c10/cuda/CUDAFunctions.cpp:109.)
  return torch._C._cuda_getDeviceCount() > 0


device(type='cpu')

In [4]:
# Data + preprocessor (identiek aan notebook 02)
preprocessor = PaddedPreprocessor()

gestures_factory = DatasetFactoryProvider.create_factory(DatasetType.GESTURES)
streamers = gestures_factory.create_datastreamer(batchsize=32, preprocessor=preprocessor)

train = streamers["train"]
valid = streamers["valid"]

trainstreamer = train.stream()
validstreamer = valid.stream()

len(train), len(valid)

2026-01-08 17:46:04.913 | INFO     | mads_datasets.base:download_data:121 - Folder already exists at /home/mischatrader1/.cache/mads_datasets/gestures
100%|██████████| 651/651 [00:00<00:00, 1379.91it/s]


(81, 20)

In [5]:
accuracy = Accuracy()

settings = TrainerSettings(
    epochs=30,  # bewust korter dan "100": dit notebook is een gerichte vergelijking
    metrics=[accuracy],
    logdir=Path("gestures_ex3"),
    train_steps=len(train),
    valid_steps=len(valid),
    reporttypes=[ReportTypes.TOML, ReportTypes.TENSORBOARD, ReportTypes.MLFLOW],
    scheduler_kwargs={"factor": 0.5, "patience": 5},
    earlystop_kwargs={
        "save": False,
        "verbose": True,
        "patience": 5,
        "delta": 0.0,
    },
)
settings

epochs: 30
metrics: [Accuracy]
logdir: gestures_ex3
train_steps: 81
valid_steps: 20
reporttypes: [<ReportTypes.TOML: 'TOML'>, <ReportTypes.TENSORBOARD: 'TENSORBOARD'>, <ReportTypes.MLFLOW: 'MLFLOW'>]
optimizer_kwargs: {'lr': 0.001, 'weight_decay': 1e-05}
scheduler_kwargs: {'factor': 0.5, 'patience': 5}
earlystop_kwargs: {'save': False, 'verbose': True, 'patience': 5, 'delta': 0.0}

In [6]:
from src_rnn.models import RNNConfig
from src_rnn.run_experiment import run_experiment

In [7]:
# # Run: Conv1D vóór GRU (experiment)
config = RNNConfig(
    input_size=3,
    hidden_size=64,
    num_layers=1,
    output_size=20,
    dropout=0.0,
)

_ = run_experiment(
    model_name="conv1d_gru_mean",                 # baseline: last-step representatie
    config=config,
    settings=settings,
    trainstreamer=train.stream(),
    validstreamer=valid.stream(),
    device=device,
    experiment_name="gestures-ex3",
    run_name="03_conv1d_gru_mean_h64_l1",
)

2026-01-08 17:46:08.750 | INFO     | mltrainer.trainer:dir_add_timestamp:24 - Logging to gestures_ex3/20260108-174608
2026-01-08 17:46:11.035 | INFO     | mltrainer.trainer:__init__:68 - Found earlystop_kwargs in settings.Set to None if you dont want earlystopping.
100%|██████████| 81/81 [00:01<00:00, 50.09it/s]
2026-01-08 17:46:12.911 | INFO     | mltrainer.trainer:report:209 - Epoch 0 train 2.7794 test 2.3943 metric ['0.2562']
100%|██████████| 81/81 [00:01<00:00, 55.23it/s]
2026-01-08 17:46:14.547 | INFO     | mltrainer.trainer:report:209 - Epoch 1 train 1.9566 test 1.5490 metric ['0.6375']
100%|██████████| 81/81 [00:01<00:00, 56.65it/s]
2026-01-08 17:46:16.148 | INFO     | mltrainer.trainer:report:209 - Epoch 2 train 1.2005 test 0.8919 metric ['0.8500']
100%|██████████| 81/81 [00:01<00:00, 58.38it/s]
2026-01-08 17:46:17.686 | INFO     | mltrainer.trainer:report:209 - Epoch 3 train 0.6940 test 0.5413 metric ['0.9688']
100%|██████████| 81/81 [00:01<00:00, 56.99it/s]
2026-01-08 17:46:1

# Resultaten & Conclusie

In dit experiment heb ik onderzocht of het toevoegen van een Conv1D-laag vóór een GRU de prestaties op gesture-classificatie verbetert.

# Vergelijking van modellen (MLflow):
- GRU (laatste timestep): val accuracy ≈ 0.97
- GRU + mean pooling: val accuracy ≈ 0.98
- Conv1D → GRU + mean pooling: val accuracy ≈ 0.99
Het Conv1D-model behaalt zowel hogere validatie-accuracy als lagere validatie-loss dan de baselines.

# Interpretatie:
De Conv1D-laag leert lokale temporele patronen (zoals korte acceleratie-pieken), waardoor de GRU een meer gestructureerde sequentierepresentatie ontvangt. Dit leidt tot snellere convergentie en betere generalisatie.

Wat werkt:
- Conv1D vóór GRU
- Mean pooling i.p.v. last timestep

Wat minder werkt:
- Alleen GRU met last-step representatie (gevoelig voor padding en timing)

# Conclusie:
Het toevoegen van Conv1D vóór een RNN is een effectieve verbetering voor dit probleem en voldoet aan de doelstelling van exercise 3.